In [1]:
import datetime
import torch

from torch_geometric.loader import DataLoader
from gnnexplainer import GNNExplainer
from pgexplainer import PGExplainer

from load_data import load_data
from gnn_dig import load_gnn_model
from train_and_evaluate_dig import train, test
from explanation_analysis import generate_explanation_dig, save_edge_masks_dig
from utils import set_seed, save_model, load_model, manipulate_dataset, parse_args

import warnings
warnings.filterwarnings('ignore')

In [2]:
args = parse_args()
# args.fast_mode = True
args.gpu_id = 0
args.dataset = "BA_2motifs"
args.manipulate_ratio = 1.
args.gnn_model = "GIN_3l_BN"
args.explainer_model = "PGExplainer"

if args.fast_mode and args.explainer_model == "PGExplainer":
    args.ex_epoch = 10

if args.dataset.lower() == "Mutagenicity".lower():
    args.save_explanation = True
    
device = torch.device(
    'cuda:%s' % args.gpu_id if torch.cuda.is_available() else 'cpu'
)
set_seed(args.seed)

print(args)

Namespace(dataset='BA_2motifs', ex_epoch=100, ex_lr=0.001, explainer_model='PGExplainer', fast_mode=False, gnn_epoch=200, gnn_hid_dim=128, gnn_lr=0.001, gnn_model='GIN_3l_BN', gpu_id=0, manipulate_ratio=1.0, save_explanation=False, seed=0)


In [3]:
# load dataset
dataset, train_dataset, val_dataset, test_dataset = load_data(args=args)

# manipulate dataset by reversing their label
if args.manipulate_ratio == 0:
    pass
else:
    train_dataset = manipulate_dataset(dataset=train_dataset, ratio=args.manipulate_ratio)
    val_dataset = manipulate_dataset(dataset=val_dataset, ratio=args.manipulate_ratio)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


Dataset: BA2MotifDataset(1000):
Number of graphs: 1000
Number of node features: 10
Number of edge features: 0
Number of classes: 2
N. train: 700, N. valid: 100, N. test: 200


In [4]:
model = load_gnn_model(args=args, dataset=dataset)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=args.gnn_lr, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

GIN_3l_BN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=10, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
  ))
  (convs): ModuleList(
    (0-1): 2 x GINConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
    ))
  )
  (relu1): Sequential(
    (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU()
  )
  (relus): ModuleList(
    (0-1): 2 x Sequential(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (readout): GlobalMeanPool()
  (ffn): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=128, out_features=2, bias=True)
  )
  (dropout): Dropout(p=0

In [5]:
best_val_acc = test_acc = 0
now = datetime.datetime.now()
best_model_filename = f"best_model_{now.hour}_{now.minute}_{now.second}.pth"

for epoch in range(1, args.gnn_epoch+1):
    loss = train(
        model=model, train_loader=train_loader, optimizer=optimizer, device=device
    )
    train_acc = test(model=model, loader=train_loader, device=device)
    val_acc = test(model=model, loader=val_loader, device=device)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_model(model=model, filename=best_model_filename)  # Save the current model
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f} Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

# Load the best model and evaluate on the test set
load_model(model=model, filename=best_model_filename)
train_acc = test(model=model, loader=train_loader, device=device)
val_acc = test(model=model, loader=val_loader, device=device)
test_acc = test(model=model, loader=test_loader, device=device)
print(f'Train Acc: {train_acc:.4f}, Valid Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 0.6936 Train Acc: 0.5029, Val Acc: 0.4900
Epoch: 002, Loss: 0.6939 Train Acc: 0.5029, Val Acc: 0.4900
Epoch: 003, Loss: 0.6942 Train Acc: 0.5029, Val Acc: 0.4900
Epoch: 004, Loss: 0.6931 Train Acc: 0.5857, Val Acc: 0.4800
Epoch: 005, Loss: 0.6926 Train Acc: 0.5029, Val Acc: 0.4900
Epoch: 006, Loss: 0.6913 Train Acc: 0.5029, Val Acc: 0.4900
Epoch: 007, Loss: 0.6919 Train Acc: 0.5729, Val Acc: 0.4700
Epoch: 008, Loss: 0.6926 Train Acc: 0.6257, Val Acc: 0.6000
Epoch: 009, Loss: 0.6881 Train Acc: 0.6400, Val Acc: 0.5300
Epoch: 010, Loss: 0.6870 Train Acc: 0.6114, Val Acc: 0.6100
Epoch: 011, Loss: 0.6828 Train Acc: 0.5043, Val Acc: 0.4900
Epoch: 012, Loss: 0.6773 Train Acc: 0.6543, Val Acc: 0.5800
Epoch: 013, Loss: 0.6677 Train Acc: 0.6814, Val Acc: 0.6100
Epoch: 014, Loss: 0.6341 Train Acc: 0.4971, Val Acc: 0.5100
Epoch: 015, Loss: 0.6323 Train Acc: 0.6657, Val Acc: 0.6000
Epoch: 016, Loss: 0.5713 Train Acc: 0.8243, Val Acc: 0.7900
Epoch: 017, Loss: 0.4766 Train Acc: 0.88

In [6]:
model.eval()
if args.explainer_model == "GNNExplainer":
    explainer = GNNExplainer(model, epochs=args.ex_epoch, lr=args.ex_lr, explain_graph=True)
elif args.explainer_model == "PGExplainer":
    explainer = PGExplainer(
        model, in_channels=args.gnn_hid_dim*2, epochs=args.ex_epoch, lr=args.ex_lr, 
        explain_graph=True, device=device
    ).to(device)
    print("training PGExplainer")
    explainer.train_explanation_network(train_dataset)
else:
    raise ValueError("%s is not a available explainer model" % args.explainer_model)

x_collector, ex_list = generate_explanation_dig(
    args=args, model=model, explainer=explainer, dataset=test_dataset, device=device
)

if args.save_explanation:
    save_edge_masks_dig(args=args, ex_list=ex_list)
    
print(f'Fidelity: {x_collector.fidelity:.4f}\n'
      f'Fidelity_inv: {x_collector.fidelity_inv:.4f}\n'
      f'Sparsity: {x_collector.sparsity:.4f}')
print(f'Train Acc: {train_acc:.4f}, Valid Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

training PGExplainer


100%|██████████| 700/700 [00:07<00:00, 88.30it/s]


Epoch: 0 | Loss: 1283.1314597129822


100%|██████████| 700/700 [00:07<00:00, 88.76it/s]


Epoch: 1 | Loss: 1259.301480859518


...

Epoch: 98 | Loss: 870.151337146759


100%|██████████| 700/700 [00:07<00:00, 88.15it/s]


Epoch: 99 | Loss: 869.7161606401205


Generate explanation for each data: 100%|██████████| 200/200 [00:03<00:00, 55.35it/s]

Fidelity: 0.0029
Fidelity_inv: 0.4645
Sparsity: 0.7678
Train Acc: 0.9943, Valid Acc: 1.0000, Test Acc: 0.0150


In [7]:
# save results
args.fidelity = round(x_collector.fidelity, 4)
args.fidelity_inv = round(x_collector.fidelity_inv, 4)
args.sparsity = round(x_collector.sparsity, 4)
args.train_acc = round(train_acc, 4)
args.val_acc = round(val_acc, 4)
args.test_acc = round(test_acc, 4)
with open('result_record.txt', 'a') as file:
    file.write('\n')
    file.write(str(args))